In [31]:
import pandas as pd
import re
from difflib import get_close_matches

# Baca file Excel
file_path = "Extracted_Text.xlsx"  # Ganti dengan nama file Anda
database_file = "../assets/databasemenu.xlsx"  # Database menu dengan kolom "Original Menu"
menu_database = pd.read_excel(database_file)
df = pd.read_excel(file_path)

# Pastikan kolom "Extracted Text" dan "File Name" ada
if {"Extracted Text", "File Name"}.issubset(df.columns):
    results = []

    # Iterasi melalui setiap baris
    for _, row in df.iterrows():
        file_name = row["File Name"]
        raw_text = row["Extracted Text"]

        if pd.notna(raw_text):  # Abaikan baris kosong
            lines = raw_text.strip().splitlines()
            menu = ""

            # Proses teks baris per baris
            for line in lines:
                if "@" in line:  # Cari baris dengan '@'
                    qty = line.strip()
                    results.append({"File Name": file_name, "Menu": menu, "Qty": qty})
                else:
                    menu = line.strip()  # Update nama menu

    # Buat DataFrame dari hasil
    df = pd.DataFrame(results, columns=["File Name", "Menu", "Qty"])

        # Pastikan database memiliki kolom 'Original Menu'
    if "Original Menu" in menu_database.columns:
        # Ambil daftar nama asli menu
        original_menus = menu_database["Original Menu"].tolist()

        # Fungsi untuk mencocokkan menu berdasarkan kemiripan
        def match_menu(menu):
            matches = get_close_matches(menu, original_menus, n=1, cutoff=0.75)  # Cutoff menentukan tingkat kemiripan minimal
            return matches[0] if matches else menu  # Jika tidak ada kecocokan, kembalikan menu asli

        # Pastikan kolom "Menu" ada dalam dataframe input
        if "Menu" in df.columns:
            # Terapkan pencocokan ke kolom Menu
            df["Corrected Menu"] = df["Menu"].apply(match_menu)
        
        df = df.drop(columns="Menu")
        df = df.rename(columns={'Corrected Menu': 'Menu'})
        df = df.loc[:,['File Name','Menu','Qty']]

        # Fungsi untuk memisahkan Qty dan Harga
        def split_qty_harga(qty_harga):
            # Menangkap Qty (angka sebelum 'x' atau '@')
            qty_match = re.match(r'(\d+)', qty_harga)
            qty = qty_match.group(1) if qty_match else ''
            
            # Menangkap Harga (angka setelah '@')
            harga_match = re.search(r'@ ([\d,]+)', qty_harga)
            harga = harga_match.group(1) if harga_match else ''
            
            return qty, harga

        # Menerapkan fungsi untuk memisahkan kolom 'Qty' dan 'Harga'
        df[['Qty', 'Harga']] = df['Qty'].apply(lambda x: pd.Series(split_qty_harga(x)))
        df['Harga']  =   df['Harga'].str.replace(',', '')
            
    # Simpan hasil ke file Excel baru
    output_file = "output1.xlsx"  # Ganti nama file sesuai keinginan
    df.to_excel(output_file, index=False)
    
    print(f"Proses selesai. Hasil disimpan di: {output_file}")
else:
    print("Kolom 'Extracted Text' atau 'File Name' tidak ditemukan di file Excel.")

Proses selesai. Hasil disimpan di: output1.xlsx


In [32]:
df

,File Name,Menu,Qty,Harga
0,SBYAMB 3.jpg,MIE GACOAN (Bks),1,13637
1,SBYAMB 3.jpg,MIE GACOAN LV 1 (Bks),2,13637
2,SBYAMB 3.jpg,MIE GACOAN LV 2 (Bks),,13637
3,SBYAMB 3.jpg,MIE GACOAN LV 4 (Bks),,13637
4,SBYAMB 3.jpg,UDANG KEJU (Bks),,12728
5,SBYAMB 3.jpg,YS FAMBUTAN (Bk),,12728
6,SBYAMB 1.jpg,TEA TARIK ICE - NP (Bks),1,6819
7,SBYAMB 2.jpg,MIE GACOAN LV 1 (Bks),1,13637
